# Matrix and Vector operation

Here, I will show the basic operations for Eigen `Matrix` and `Vector`. The arithmetic is based upon C++ operator overloads such as `+`, `-` and `*`. Moreover, proper operations for linear algebra like `dot()`, `cross()`, etc are provided. For `Matrix` class, only operation which makes sense in Linear Algebra are allowed. For example, the sum of a scalar with a vector is not allowed. To perform general operations in a coefficient-wise manner, Eigen provides the `Array` class, which can be seem as a general-purpose arrays. This class will be explored in another notebook.

As in previous notebooks, the material presented here is highly based on the [Eigen's official documentation](https://eigen.tuxfamily.org/dox/group__TutorialMatrixArithmetic.html), except for few modifications and adaptations.

Before anything, let's include the necessary headers and Eigen namespace:

In [1]:
#include <iostream>
#include <eigen3/Eigen/Dense>

using namespace Eigen;

## Addition and subtraction

To perform addition and subtraction operations, the left and right size must have the same number of rows and columns, as well as the same `Scalar` type. Currently, there is no automatic promotion implemented in Eigen. The available operators are:

* Binary operator `+` as in `a + b`;
* Binary operator `-` as in `a - b`;
* Unary operator `-` as in `-a`;
* Compound operator `+=` as in `a += b`;
* Compound operator `-=` as in `a -= b`.

Let's see some examples.

In [2]:
Matrix2d mat1;
mat1 << 1, 2,
        3, 4;

MatrixXd mat2(2, 2);
mat2 << 2, 3,
        1, 4;

Simple addition:

In [3]:
std::cout << "mat1 + mat2 =\n" << mat1 + mat2;

mat1 + mat2 =
3 5
4 8

Subtraction:

In [4]:
std::cout << "mat1 - mat2 =\n" << mat1 - mat2;

mat1 - mat2 =
-1 -1
 2  0

Cummulative operator:

In [5]:
std::cout << "Doing mat1 += mat2\n";

mat1 += mat2;

std::cout << "Updated mat1 =\n" << mat1;

Doing mat1 += mat2
Updated mat1 =
3 5
4 8

Similar is valid for vectors, see:

In [6]:
Vector3d v, w;
v << 1, 2, 3;
w << 1, 0, 0;

std::cout << "-v + w - v =\n" << -v + w - v;

-v + w - v =
-1
-4
-6

## Multiplication and division by a scalar

You can multiplicate a `Matrix` by a scalar, as well as perform some other operations. In summary, the available operations are:

* Commutative binary operator `*` as in `scalar * matrix`;
* Non-commutative binary operator `/` as in `matrix / scalar`;
* Non-commutative compound operator `*=` as in `matrix *= scalar`;
* Non-commutative compound operator `/=` as in `matrix /= scalar`;

Some examples are shown below.

In [7]:
Matrix2d mat3;
mat3 << 1, 2,
        3, 4;

Vector3d vec1(1, 2, 3);

A `Matrix` multiplied by scalar:

In [8]:
std::cout << "mat3 * 2.5 =\n" << mat3 * 2.5;

mat3 * 2.5 =
2.5   5
7.5  10

A vector multiplied by a scalar:

In [9]:
std::cout << "0.1 * vec1 =\n" << 0.1 * vec1;

0.1 * vec1 =
0.1
0.2
0.3

Cummulative multiplication:

In [10]:
vec1 *= 2;

std::cout << "Updated vec1 =\n" << vec1;

Updated vec1 =
2
4
6

A remark: caution is need when using the operators show above. They are valid only for `Matrix` and scalar. For instance, `vec1 *= vec1` is not possible. Moreover, watch out the non-commutative ones!

## Quick note about expression templates

This is an advanced topic, but it is worthy to mention here. 

In Eigen, all operations don't perform any computation by themselves, until the whole expression is evaluated, usually in `operator=`. This is used for optimize the code compilation. Any modern optimizing compiler is capable to handle it. For example, one can simply do:

```C++
VectorXf a(50), b(50), c(50), d(50);
a = 3 * b + 4 * c + 5 * d;
```

Note that there is no need to perform the loop by hand! The compiler will do it for you. Behind the scenes, desconsidering SIMD instructions, the compiler will do a loop like this:

```C++
for (int i = 0; i < 50; ++i) a[i] = 3 * b[i] + 4 * c[i] + 5 * d[i];
```

Hence, no need to worry, let it be!

## Transposition, conjugation and adjoint

All the transposition, conjugation and adjoint of a `Matrix` object (matrices and vectors) are performed by member functions `transpose()`, `conjugate()` and `adjoint()`, respectively. See below some examples: 

Let's define a complex random `Matrix`:

In [11]:
MatrixXcf matc = MatrixXcf::Random(2, 2);

std::cout << "The matrix matc:\n" << matc;

The matrix matc:
(0.254622,-0.606912) (-0.187187,0.783351)
(-0.431943,-0.74263) (0.183727,-0.315103)

The transpose:

In [12]:
std::cout << "matc transposed:\n" << matc.transpose();

matc transposed:
(0.254622,-0.606912) (-0.431943,-0.74263)
(-0.187187,0.783351) (0.183727,-0.315103)

The conjugation:

In [13]:
std::cout << "matc conjugation:\n" << matc.conjugate();

matc conjugation:
  (0.254622,0.606912) (-0.187187,-0.783351)
  (-0.431943,0.74263)   (0.183727,0.315103)

The adjoint:

In [14]:
std::cout << "matc adjoint:\n" << matc.adjoint();

matc adjoint:
  (0.254622,0.606912)   (-0.431943,0.74263)
(-0.187187,-0.783351)   (0.183727,0.315103)

Remember that for Real matrix, the conjugation is the matrix itself! Thus, the `adjoint()` is equivalent to `transpose()`.

Another important remark is concerning __aliasing issue__. As you have a right-hand side expression assigned to a _lvalue_ variable, this operation is initialized before the evaluation! So, if you do `a = a.transpose()`, it will begin to write the results on `a` before the evaluation of the transpose is complete. So, doing `a = a.transpose()` you will not replace `a` by its transpose! If you want to circumvent such behavior, use the `transposeInPlace()`. Analogously, you could use `adjointInPlace()`.

## Matrix-matrix and matrix-vector multiplications

To perform matrix-matrix multiplications, the `operator*` is used. Vectors are a special case of matrices, so it will be handled as well. Let's see some examples below:

First, let's declare a matrix and two vectors:

In [15]:
Matrix2d mat4;
mat4 << 1, 2,
        3, 4;

Vector2d vec_u(-1, 1), vec_v(2, 0);

* A matrix-matrix multiplication:

In [16]:
std::cout << "mat4 * mat4 =\n" << mat4 * mat4;

mat4 * mat4 =
 7 10
15 22

Note that this the classical matricial product. The lines of the left-hand side matrix are multiplied by the columns of right-hand side matrix.

* A matrix-vector multiplication:

In [17]:
std::cout << "mat4 * vec_u =\n" << mat4 * vec_u;

mat4 * vec_u =
1
1

This is the classical application of a matrix (linear operator) on a vector, generating a vector, just like $Ax = b$.

* A vector-vector multiplication:

In [18]:
std::cout << "vec_u * vec_v^T =\n" << vec_u * vec_v.transpose();

vec_u * vec_v^T =
-2 -0
 2  0

That is a somewhat unexpected behavior! You must recall that all `VectorXt`s are essentially `Matrix`, so the same multiplication rules are applied. Furthermore, always remember that vectors in Eigen are column-vectors! Hence, to obtain a proper multiplication in the sense of linear algebra, we can do it as

In [19]:
std::cout << "vec_u^T * vec_v =\n" << vec_u.transpose() * vec_v;

vec_u^T * vec_v =
-2

A remarkable feature of the matrix-matrix multiplication is that the aliasing issue is automatically circumvented. Eigen treats such multiplications as special cases! So, you can do something like this:

In [20]:
mat4 = mat4 * mat4;

std::cout << mat4;

 7 10
15 22

Behind the scenes, Eigen creates a temporary object for `Matrix`, so no need to worry!

## Dot and cross product

We will use the previous vectors. So let's remember their values:

In [21]:
std::cout << "vec_u =\n" << vec_u << std::endl;
std::cout << "vec_v =\n" << vec_v << std::endl;

vec_u =
-1
 1
vec_v =
2
0


Now we'll see some action:

* Dot product:

In [22]:
std::cout << "Dot product vec_u by vec_v: " << vec_u.dot(vec_v);

Dot product vec_u by vec_v: -2

which is, as one must expect, the same as `vec_u^T * vec_v`.

* Cross product:

For this one, we need 3D vectors, since this is the space where cross product is well defined.

In [23]:
Vector3d vec_w(1, 2, 3), vec_z(0, 1, 2);

std::cout << "Cross product vec_w by vec_z:\n" << vec_w.cross(vec_z);

Cross product vec_w by vec_z:
 1
-2
 1

An important point: In Tensor Algebra, dot product are defined in high order tensors as well. In Eigen, using only `Matrix` makes the aforementioned operation unavailable in the tensor algebra sense.

## Basic arithmetic reduction operations

Some convenience reduction operations are provided be Eigen. This operations reduces a matrix or vector to a single value, such as the sum of the entries of a vector, or the maximum coefficient value in a matrix. Let's see this operations below.

In [24]:
Matrix2d mat5;
mat5 << 1, 2, 3, 4;

* Cummulative sum of the entries:

In [25]:
std::cout << "mat5.sum() = " << mat5.sum();

mat5.sum() = 10

* Cummulative product of the entries:

In [26]:
std::cout << "vec_w.prod() = " << vec_w.prod();

vec_w.prod() = 6

* Mean of the entry values in a `Matrix` or `Vector`:

In [27]:
std::cout << "mat5.mean() = " << mat5.mean();

mat5.mean() = 2.5

* Minimum coefficient value:

In [28]:
std::cout << "vec_u.minCoeff() = " << vec_u.minCoeff();

vec_u.minCoeff() = -1

* Maximum coefficient value:

In [29]:
std::cout << "vec_v.maxCoeff() = " << vec_v.maxCoeff() << std::endl;
std::cout << "mat5.maxCoeff() = " << mat5.maxCoeff();

vec_v.maxCoeff() = 2
mat5.maxCoeff() = 4

* Trace of a Matrix (the sum of the values on the diagonal):

In [30]:
std::cout << "mat5.trace() = " << mat5.trace();

mat5.trace() = 5

## Validity of Operations

By means of internal assertings, Eigen checks the validity of operations and compatibility between operands. When is possible, it is check at compile time. In some cases, specially when using dynamic-size `Matrix`, compile time check is not possible, so the check is performed in run-time, using assertions. Let's see two quickly examples:

* Compile time check:
```C++
Matrix3f m;
Matrix4f v;
// ...some calculations here...
v = m * v;  // Compile-time error! Different sizes.
```

* Run-time check:
```C++
MatrixXf m(3,3);
VectorXf v(4);
v = m * v;  // Run-time error! Incompatible dimensions for product.
```

For further details, check it [here](https://eigen.tuxfamily.org/dox/TopicAssertions.html).